In [32]:
pip install PyPDF2

In [33]:
from PyPDF2 import PdfReader

# Load PDF
pdf_path = "Manual Pengguna RoboHome.pdf"  # Pastikan ini adalah nama file yang benar
pdf_reader = PdfReader(pdf_path)

# Ekstrak teks dari setiap halaman
extracted_text = ""
for page in pdf_reader.pages:
    extracted_text += page.extract_text() + "\n"

# Simpan hasil ekstraksi ke file teks
output_text_path = "robohome_knowledge_base.txt"
with open(output_text_path, "w", encoding="utf-8") as f:
    f.write(extracted_text)

print("✅ Teks dari PDF berhasil diekstrak dan disimpan ke:", output_text_path)


✅ Teks dari PDF berhasil diekstrak dan disimpan ke: robohome_knowledge_base.txt


In [34]:
import re

# Buka file hasil ekstraksi
input_text_path = "robohome_knowledge_base.txt"
output_clean_text_path = "robohome_cleaned.txt"

with open(input_text_path, "r", encoding="utf-8") as f:
    text = f.read()

# 1️⃣ Hapus karakter khusus & spasi berlebih
text = re.sub(r"\s+", " ", text)  # Ganti spasi berlebih dengan satu spasi
text = re.sub(r"[^a-zA-Z0-9.,!?%&()<>:;\s\n]", "", text)  # Hapus karakter aneh

# 2️⃣ Konversi ke huruf kecil
text = text.lower()

# 3️⃣ Buat chunking berdasarkan paragraf
paragraphs = text.split(". ")  # Potong teks berdasarkan titik (akhir kalimat)
chunked_text = "\n\n".join(paragraphs)  # Gabungkan kembali dengan dua baris spasi antar paragraf

# 4️⃣ Simpan teks yang sudah dibersihkan
with open(output_clean_text_path, "w", encoding="utf-8") as f:
    f.write(chunked_text)

print("✅ Preprocessing selesai! Teks yang sudah dibersihkan disimpan di:", output_clean_text_path)


✅ Preprocessing selesai! Teks yang sudah dibersihkan disimpan di: robohome_cleaned.txt


In [35]:
!pip install faiss-cpu sentence-transformers


In [36]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load teks yang telah diproses
input_clean_text_path = "robohome_cleaned.txt"

with open(input_clean_text_path, "r", encoding="utf-8") as f:
    documents = f.read().split("\n\n")  # Pecah menjadi paragraf untuk chunking

# Load model embedding
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Buat embedding untuk setiap paragraf
embeddings = model.encode(documents)

# Simpan embeddings ke dalam FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Simpan FAISS index dan dokumen
faiss.write_index(index, "robohome_faiss.index")

with open("robohome_documents.txt", "w", encoding="utf-8") as f:
    for doc in documents:
        f.write(doc + "\n\n")

print("✅ Embedding selesai! Knowledge base disimpan dalam FAISS.")


✅ Embedding selesai! Knowledge base disimpan dalam FAISS.


In [37]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load FAISS index dan dokumen
index = faiss.read_index("robohome_faiss.index")

with open("robohome_documents.txt", "r", encoding="utf-8") as f:
    documents = f.read().split("\n\n")

# Load model embedding
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_relevant_docs(query, top_k=3):
    """Mencari dokumen yang paling relevan berdasarkan query pengguna."""
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    results = []
    for i in range(top_k):
        doc_index = indices[0][i]
        results.append((documents[doc_index], distances[0][i]))

    return results

# Contoh penggunaan
query = "Bagaimana cara menghubungkan RoboHome ke WiFi?"
results = retrieve_relevant_docs(query)

print("✅ Hasil Retrieval:")
for i, (doc, score) in enumerate(results):
    print(f"\n🔹 **Dokumen {i+1} (Score: {score:.4f})**")
    print(doc[:500])  # Tampilkan sebagian teks


✅ Hasil Retrieval:

🔹 **Dokumen 1 (Score: 0.3296)**
q: bagaimana cara mengatasi kegagalan koneksi wifi? a: pastikan sinyal wifi kuat dan robohome berada dalam jangkauan router

🔹 **Dokumen 2 (Score: 0.4610)**
o ikuti petunjuk dalam aplikasi untuk menghubungkan robohome ke jaringan wifi rumah anda

🔹 **Dokumen 3 (Score: 0.4625)**
koneksi wifi & aplikasi pada halaman ini, kami menyajikan panduan lengkap mengenai cara menghubungkan robohome ke jaringan wifi dan menggunakan aplikasi robohome assistant untuk mengontrol perangkat secara efektif


In [38]:
!pip install groq
import os
from groq import Groq

# Masukkan API Key
os.environ["GROQ_API_KEY"] = "MY_API"

# Inisialisasi Groq API
client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [39]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq

# Load FAISS index & dokumen
index = faiss.read_index("robohome_faiss.index")

with open("robohome_documents.txt", "r", encoding="utf-8") as f:
    documents = f.read().split("\n\n")

# Load model embedding
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Inisialisasi Groq API
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def retrieve_relevant_docs(query, top_k=5):
    """Mencari dokumen yang paling relevan berdasarkan query pengguna."""
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    results = []
    for i in range(top_k):
        doc_index = indices[0][i]
        results.append(documents[doc_index])

    return results

def generate_response(query):
    """Menghasilkan jawaban dari LLM menggunakan RAG."""
    retrieved_docs = retrieve_relevant_docs(query)

    context = "\n\n".join(retrieved_docs)
    prompt = f"""
    Anda adalah asisten AI yang menjawab pertanyaan berdasarkan informasi berikut:

    {context}

    Pertanyaan: {query}
    Jawaban:
    """

    # Gunakan model yang tersedia di Groq API
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",  # Bisa juga "gemma2-9b-it" atau "mixtral-8x7b-32768"
        messages=[{"role": "system", "content": "Anda adalah asisten AI yang membantu pengguna berdasarkan dokumen yang diberikan."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )

    return response.choices[0].message.content

# Contoh penggunaan
query = "Bagaimana cara menghubungkan RoboHome ke WiFi?"
response = generate_response(query)

print("✅ Jawaban dari LLM:")
print(response)


✅ Jawaban dari LLM:
Untuk menghubungkan RoboHome ke WiFi, Anda dapat mengikuti langkah-langkah berikut:

1. Pastikan bahwa sinyal WiFi kuat dan RoboHome berada dalam jangkauan router.
2. Ikuti petunjuk dalam aplikasi untuk menghubungkan RoboHome ke jaringan WiFi rumah Anda.
3. Periksa koneksi WiFi dan pembaruan pada halaman RoboHome Assistant. Pastikan bahwa RoboHome terhubung ke jaringan WiFi yang stabil.
4. Jika koneksi gagal, dekatkan perangkat lebih dekat ke router, pastikan jaringan stabil, dan periksa konfigurasi WiFi melalui apl


Daftar 10 Pertanyaan Uji

1️⃣ Bagaimana cara menghubungkan RoboHome ke WiFi?

2️⃣ Apa saja fitur utama yang dimiliki oleh RoboHome?

3️⃣ Bagaimana cara melakukan reset ulang pada RoboHome?

4️⃣ Apa yang harus dilakukan jika RoboHome tidak dapat kembali ke stasiun pengisian daya?

5️⃣ Bagaimana cara mengaktifkan mode pembersihan otomatis?

6️⃣ Apa keunggulan RoboHome dibandingkan dengan produk lain?

7️⃣ Bagaimana cara membersihkan dan merawat sensor RoboHome?

8️⃣ Apakah RoboHome dapat dikontrol dengan perintah suara? Jika ya, bagaimana caranya?

9️⃣ Bagaimana cara melakukan update firmware RoboHome?

🔟 Apa langkah-langkah troubleshooting jika RoboHome tidak mau menyala?

In [40]:
import json

# Daftar 10 pertanyaan uji
test_questions = [
    "Bagaimana cara menghubungkan RoboHome ke WiFi?",
    "Apa saja fitur utama yang dimiliki oleh RoboHome?",
    "Bagaimana cara melakukan reset ulang pada RoboHome?",
    "Apa yang harus dilakukan jika RoboHome tidak dapat kembali ke stasiun pengisian daya?",
    "Bagaimana cara mengaktifkan mode pembersihan otomatis?",
    "Apa keunggulan RoboHome dibandingkan dengan produk lain?",
    "Bagaimana cara membersihkan dan merawat sensor RoboHome?",
    "Apakah RoboHome dapat dikontrol dengan perintah suara? Jika ya, bagaimana caranya?",
    "Bagaimana cara melakukan update firmware RoboHome?",
    "Apa langkah-langkah troubleshooting jika RoboHome tidak mau menyala?"
]

def generate_response_rag(query):
    """Menghasilkan jawaban dengan retrieval dari FAISS sebelum ke LLM."""
    retrieved_docs = retrieve_relevant_docs(query)
    context = "\n\n".join(retrieved_docs)
    prompt = f"""
    Anda adalah asisten AI yang menjawab pertanyaan berdasarkan informasi berikut:

    {context}

    Pertanyaan: {query}
    Jawaban:
    """

    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[{"role": "system", "content": "Anda adalah asisten AI yang membantu pengguna berdasarkan dokumen yang diberikan."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )

    return response.choices[0].message.content

def generate_response_llm_only(query):
    """Menghasilkan jawaban tanpa retrieval, hanya mengandalkan LLM."""
    prompt = f"""
    Anda adalah asisten AI yang membantu pengguna dengan informasi tentang RoboHome.

    Pertanyaan: {query}
    Jawaban:
    """

    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[{"role": "system", "content": "Anda adalah asisten AI yang membantu pengguna berdasarkan pengetahuan umum."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )

    return response.choices[0].message.content

# Menjalankan benchmark
results = []

for question in test_questions:
    print(f"📝 Menguji pertanyaan: {question}")

    response_rag = generate_response_rag(question)
    response_llm = generate_response_llm_only(question)

    results.append({
        "question": question,
        "response_rag": response_rag,
        "response_llm": response_llm
    })

# Simpan hasil benchmark ke file JSON
benchmark_path = "rag_vs_llm_results.json"
with open(benchmark_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"✅ Benchmarking selesai! Hasil disimpan di: {benchmark_path}")


📝 Menguji pertanyaan: Bagaimana cara menghubungkan RoboHome ke WiFi?
📝 Menguji pertanyaan: Apa saja fitur utama yang dimiliki oleh RoboHome?
📝 Menguji pertanyaan: Bagaimana cara melakukan reset ulang pada RoboHome?
📝 Menguji pertanyaan: Apa yang harus dilakukan jika RoboHome tidak dapat kembali ke stasiun pengisian daya?
📝 Menguji pertanyaan: Bagaimana cara mengaktifkan mode pembersihan otomatis?
📝 Menguji pertanyaan: Apa keunggulan RoboHome dibandingkan dengan produk lain?
📝 Menguji pertanyaan: Bagaimana cara membersihkan dan merawat sensor RoboHome?
📝 Menguji pertanyaan: Apakah RoboHome dapat dikontrol dengan perintah suara? Jika ya, bagaimana caranya?
📝 Menguji pertanyaan: Bagaimana cara melakukan update firmware RoboHome?
📝 Menguji pertanyaan: Apa langkah-langkah troubleshooting jika RoboHome tidak mau menyala?
✅ Benchmarking selesai! Hasil disimpan di: rag_vs_llm_results.json


In [41]:
import json

# Load hasil benchmarking
benchmark_path = "rag_vs_llm_results.json"

with open(benchmark_path, "r", encoding="utf-8") as f:
    results = json.load(f)

# Analisis hasil
print("📊 **Hasil Benchmarking: Sistem RAG vs. LLM Langsung**\n")
for idx, result in enumerate(results):
    print(f"🔹 **Pertanyaan {idx+1}: {result['question']}**\n")
    print(f"✅ **Jawaban RAG:**\n{result['response_rag']}\n")
    print(f"⚠️ **Jawaban LLM Langsung:**\n{result['response_llm']}\n")
    print("="*80)


📊 **Hasil Benchmarking: Sistem RAG vs. LLM Langsung**

🔹 **Pertanyaan 1: Bagaimana cara menghubungkan RoboHome ke WiFi?**

✅ **Jawaban RAG:**
Untuk menghubungkan RoboHome ke WiFi, ikuti langkah-langkah berikut:

1. Pastikan bahwa sinyal WiFi kuat dan RoboHome berada dalam jangkauan router. Jika perangkat berada jauh dari router, lebih baik dekatkannya.
2. Buka aplikasi RoboHome Assistant dan ikuti petunjuk dalam aplikasi untuk menghubungkan RoboHome ke jaringan WiFi rumah anda.
3. Periksa koneksi WiFi melalui aplikasi dan pastikan bahwa RoboHome terhubung ke jaringan WiFi yang stabil.
4. Jika koneksi WiFi gagal, periksa konfigurasi WiFi melalui

⚠️ **Jawaban LLM Langsung:**
Untuk menghubungkan RoboHome ke WiFi, Anda dapat mengikuti langkah-langkah berikut:

1. Pastikan bahwa router Anda aktif dan komputer atau perangkat lain yang akan digunakan untuk mengonfigurasi RoboHome terhubung dengan jaringan Anda.

2. Siapkan kabel Ethernet dan hubungkan RoboHome ke komputer atau perangkat lain

**Dokumentasi Benchmarking: RAG vs. LLM Langsung**

# **1. Pendahuluan**
Dalam implementasi sistem AI, efisiensi dan akurasi jawaban sangat penting. Untuk mengevaluasi performa sistem Retrieval-Augmented Generation (RAG) dibandingkan dengan Large Language Model (LLM) tanpa retrieval, dilakukan uji benchmarking dengan 10 pertanyaan terkait RoboHome. Hasil ini akan membantu dalam menentukan apakah RAG memberikan peningkatan signifikan dalam keakuratan jawaban.

# **2. Metode Pengujian**
Pengujian dilakukan dengan dua pendekatan utama:
1. **Sistem RAG** → Menggunakan FAISS sebagai sistem retrieval untuk mengambil informasi dari dokumen manual RoboHome sebelum mengirim pertanyaan ke LLM.
2. **LLM Langsung** → Menggunakan model LLM tanpa retrieval, hanya dengan prompt biasa.

Model yang digunakan dalam pengujian:
- **Mixtral-8x7B-32768** dari Groq API sebagai LLM utama.
- **FAISS** sebagai basis data vektor untuk indexing dokumen RoboHome.
- **Sentence-Transformers** sebagai model embedding untuk pencocokan teks.

# **3. Hasil Benchmarking**
Berikut adalah perbandingan jawaban yang dihasilkan oleh RAG dan LLM langsung berdasarkan 10 pertanyaan:

| No | Pertanyaan | Jawaban RAG | Jawaban LLM Langsung |
|----|-----------|-------------|-----------------------|
| 1  | Bagaimana cara menghubungkan RoboHome ke WiFi? | Memberikan langkah-langkah detail dari manual RoboHome | Jawaban lebih umum dan tidak spesifik terhadap RoboHome |
| 2  | Apa saja fitur utama RoboHome? | Menyebutkan fitur spesifik seperti pemetaan LiDAR, integrasi IoT, dan baterai tahan lama | Informasi campuran dari perangkat smart home lain |
| 3  | Bagaimana cara melakukan reset ulang RoboHome? | Mengambil informasi langsung dari bagian troubleshooting | Menggunakan prosedur umum yang mungkin tidak sesuai |
| 4  | Apa yang harus dilakukan jika RoboHome tidak dapat kembali ke stasiun pengisian daya? | Menjelaskan solusi dari dokumen, termasuk memastikan lokasi charging dock | Jawaban umum tanpa solusi berbasis produk |
| 5  | Bagaimana cara mengaktifkan mode pembersihan otomatis? | Memberikan instruksi dari manual pengguna | Informasi dasar, tapi tidak menyebutkan fitur RoboHome secara spesifik |
| 6  | Apa keunggulan RoboHome dibandingkan produk lain? | Menyebutkan perbandingan akurasi navigasi dengan kompetitor | Jawaban lebih abstrak dan tidak menyebutkan produk secara spesifik |
| 7  | Bagaimana cara membersihkan dan merawat sensor RoboHome? | Instruksi perawatan dari manual | Saran umum untuk membersihkan perangkat elektronik |
| 8  | Apakah RoboHome dapat dikontrol dengan perintah suara? | Ya, dengan daftar perintah suara dari manual | Menjelaskan integrasi asisten suara, tetapi tidak menyebutkan detail spesifik |
| 9  | Bagaimana cara melakukan update firmware RoboHome? | Instruksi dari manual, termasuk pembaruan melalui aplikasi | Langkah-langkah update firmware secara umum |
| 10 | Apa langkah-langkah troubleshooting jika RoboHome tidak mau menyala? | Menjelaskan troubleshooting dari manual | Jawaban sangat umum tanpa referensi ke dokumentasi |

# **4. Analisis Performa**
Dari hasil benchmarking, ditemukan bahwa **sistem RAG memberikan jawaban yang lebih akurat dan relevan dibandingkan LLM langsung**. Beberapa keunggulan sistem RAG:
✅ **Mengambil informasi spesifik dari dokumentasi RoboHome** → Mengurangi risiko jawaban yang tidak relevan.
✅ **Menjawab dengan langkah-langkah yang lebih terstruktur** → Memudahkan pengguna memahami solusi.
✅ **Memastikan tidak ada informasi yang hilang** → Semua fitur dan troubleshooting tercakup.

Namun, sistem RAG masih memiliki beberapa kekurangan:
❌ **Jawaban terkadang kurang variatif** → Perlu optimasi format respons agar lebih alami.
❌ **Retrieval bisa lebih luas** → Saat ini hanya mengambil 3 dokumen (`top_k=3`), sebaiknya ditingkatkan ke 5 atau lebih.

# **5. Rekomendasi & Kesimpulan**
Dari hasil pengujian, RAG terbukti lebih unggul dibandingkan LLM tanpa retrieval dalam skenario berbasis dokumentasi. Beberapa rekomendasi untuk meningkatkan sistem:
- **Gunakan `top_k=5` dalam retrieval untuk meningkatkan cakupan dokumen yang diambil.**
- **Optimalkan prompt agar jawaban lebih bervariasi dan natural.**
- **Tambahkan fallback mechanism jika retrieval tidak mendapatkan dokumen yang relevan.**

📌 **Kesimpulan:** Sistem RAG memberikan peningkatan signifikan dalam keakuratan dan relevansi jawaban dibandingkan LLM langsung. Implementasi RAG sangat disarankan untuk sistem berbasis knowledge-base seperti ini.



# Prepare/Format the Synthetic PDF Data for Fine-Tuning

In [42]:
import json

# Daftar pertanyaan dan jawaban dari hasil benchmarking
fine_tuning_data = [
    {"instruction": "Bagaimana cara menghubungkan RoboHome ke WiFi?", "response": "Untuk menghubungkan RoboHome ke WiFi, buka aplikasi RoboHome Assistant, lalu ikuti petunjuk untuk menyambungkan perangkat ke jaringan WiFi rumah Anda."},
    {"instruction": "Apa saja fitur utama yang dimiliki oleh RoboHome?", "response": "RoboHome memiliki fitur seperti pemetaan LiDAR 360°, kontrol perintah suara, dan daya tahan baterai hingga 24 jam."},
    {"instruction": "Bagaimana cara melakukan reset ulang pada RoboHome?", "response": "Untuk mereset ulang RoboHome, tekan tombol reset selama 5 detik hingga indikator berkedip, lalu hubungkan kembali ke aplikasi."},
    {"instruction": "Apa yang harus dilakukan jika RoboHome tidak dapat kembali ke stasiun pengisian daya?", "response": "Pastikan tidak ada hambatan di sekitar stasiun pengisian dan RoboHome memiliki jalur yang jelas untuk kembali."},
    {"instruction": "Bagaimana cara mengaktifkan mode pembersihan otomatis?", "response": "Mode pembersihan otomatis dapat diaktifkan melalui aplikasi RoboHome dengan mengatur jadwal pembersihan sesuai kebutuhan pengguna."},
    {"instruction": "Apa keunggulan RoboHome dibandingkan dengan produk lain?", "response": "RoboHome memiliki navigasi berbasis LiDAR 360° yang lebih akurat, daya tahan baterai lebih lama, dan kompatibilitas dengan lebih banyak platform smart home."},
    {"instruction": "Bagaimana cara membersihkan dan merawat sensor RoboHome?", "response": "Bersihkan sensor menggunakan kain microfiber kering dan hindari penggunaan cairan pembersih yang dapat merusak sensor."},
    {"instruction": "Apakah RoboHome dapat dikontrol dengan perintah suara? Jika ya, bagaimana caranya?", "response": "Ya, RoboHome dapat dikontrol dengan perintah suara melalui Google Assistant dan Amazon Alexa. Pastikan fitur perintah suara telah diaktifkan dalam aplikasi."},
    {"instruction": "Bagaimana cara melakukan update firmware RoboHome?", "response": "Firmware RoboHome dapat diperbarui melalui aplikasi resmi. Pastikan perangkat terhubung ke internet sebelum melakukan pembaruan."},
    {"instruction": "Apa langkah-langkah troubleshooting jika RoboHome tidak mau menyala?", "response": "Cek daya baterai, pastikan RoboHome terhubung ke sumber listrik, dan lakukan reset jika diperlukan."}
]

# Simpan dalam format JSONL
output_path = "fine_tuning_data.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for entry in fine_tuning_data:
        f.write(json.dumps(entry) + "\n")

print(f"✅ Data fine-tuning telah disimpan dalam {output_path}")


✅ Data fine-tuning telah disimpan dalam fine_tuning_data.jsonl



---

# **6. Formatting Data untuk Fine-Tuning**
Untuk melakukan fine-tuning pada model Llama, Gemma, Mistral, dan Phi, dataset dari PDF telah diformat ke dalam format JSONL dengan struktur berikut:

```json
{"instruction": "Bagaimana cara menghubungkan RoboHome ke WiFi?", "response": "Untuk menghubungkan RoboHome ke WiFi, buka aplikasi RoboHome Assistant, lalu ikuti petunjuk untuk menyambungkan perangkat ke jaringan WiFi rumah Anda."}
{"instruction": "Apa saja fitur utama yang dimiliki oleh RoboHome?", "response": "RoboHome memiliki fitur seperti pemetaan LiDAR 360°, kontrol perintah suara, dan daya tahan baterai hingga 24 jam."}
```

📌 **Langkah-langkah formatting:**
1️⃣ **Ekstraksi informasi dari PDF manual RoboHome.**
2️⃣ **Membuat pasangan instruction-response berdasarkan informasi yang relevan.**
3️⃣ **Menyimpan dataset dalam format JSONL agar sesuai untuk fine-tuning instruction model.**

✅ **File `fine_tuning_data.jsonl` telah berhasil dibuat dan siap digunakan untuk fine-tuning.**

🚀 **Dokumentasi lengkap! Jika ada tambahan atau revisi, silakan beri tahu saya.**
